### Umgebung installieren
- Power Shell
- Eingabe: pip install requests beautifulsoup4 pandas

### Daten von Website holen
url =  https://www.scrapethissite.com/pages/forms/
- Alle Seiten durchgehen & aus jeder Tabellenzeile auslesen
- Schleife erstellen: Hole Seite 1 => lese alle teams => speichere => gehe zu Seite 2 => ...bis zur letzten Seite



In [1]:
# benötigten Bibliotheken runterladen

import time
import requests
import pandas as pd
from bs4 import BeautifulSoup

# Basis URL von der Übungsseite (Hockey-Teams)
BASE_URL = "https://www.scrapethissite.com/pages/forms/"

# Headers mit User Agent
HEADERS = {"User-Agent": "Mozilla/5.0 (NHL-Scraper-Student/1.0)"}


In [2]:
# Neue Seite laden & in BeautifulSoup übersetzen

def get_page(page_number: int = 1):
  response = requests.get(BASE_URL, params={"page": page_number}, # params... hängt ?page=1 & 2 an URL
                          timeout=30) # timeout = nicht ewig hängen, wenn seite nicht antwortet
  response.raise_for_status() # wirft fehler, falls server z.b. 404/500 zurückgibt
  return BeautifulSoup(response.text, "html.parser") # wandelt HTML in Soup-Objekt um

In [3]:
# Wie viele Seiten gibt es (Pagination)
def get_total_pages(soup: BeautifulSoup):
    max_page = 1
    for a in soup.select("ul.pagination li a"): # CSS-Selektor
        text = a.get_text(strip=True)
        if text.isdigit(): # stellt sicher, dass es Zahl ist
            max_page = max(max_page, int(text))
    return max_page

first_soup = get_page(1)
total_pages = get_total_pages(first_soup)
print("Seiten insgesamt:", total_pages)


Seiten insgesamt: 24


In [4]:
# Eine Tabellen-Seite auslesen

def extract_rows_from_soup(soup: BeautifulSoup):
    rows = []
    for tr in soup.select("tr.team"): # Jede Zeile entspricht einem Datensatz
        name = tr.select_one("td.name").get_text(strip=True)
        year = tr.select_one("td.year").get_text(strip=True)
        wins = tr.select_one("td.wins").get_text(strip=True)
        losses = tr.select_one("td.losses").get_text(strip=True)
        ot_losses_cell = tr.select_one("td.ot-losses")
        ot_losses = ot_losses_cell.get_text(strip=True) if ot_losses_cell else ""
        pct = tr.select_one("td.pct").get_text(strip=True)
        gf = tr.select_one("td.gf").get_text(strip=True)
        ga = tr.select_one("td.ga").get_text(strip=True)
        diff = tr.select_one("td.diff").get_text(strip=True)

        rows.append({
            "Team Name": name,
            "Year": year,
            "Wins": wins,
            "Losses": losses,
            "OT Losses": ot_losses,
            "Win %": pct,
            "Goals For (GF)": gf,
            "Goals Against (GA)": ga,
            "+ / -": diff,
        })
    return rows


In [5]:
# Alle Seiten einsammeln 

all_rows = []

# 1) Normale Variante: wir kennen total_pages (aus der Pagination)
if total_pages > 1:
    for page in range(1, total_pages + 1):
        soup = get_page(page)
        rows = extract_rows_from_soup(soup)
        all_rows.extend(rows)
        print(f"Seite {page}/{total_pages}: {len(rows)} Zeilen")
        time.sleep(0.3)  # kurze Pause

# 2) Fallback: Falls Pagination nichts geliefert hat, blättern bis keine Zeilen mehr kommen
else:
    page = 1
    while True:
        soup = get_page(page)
        rows = extract_rows_from_soup(soup)
        if not rows:
            print(f"An Seite {page} gab es keine Daten mehr. Stop.")
            break
        all_rows.extend(rows)
        print(f"Seite {page}: {len(rows)} Zeilen")
        page += 1
        time.sleep(0.3)

print("Gesamt gesammelte Zeilen:", len(all_rows))


Seite 1/24: 25 Zeilen
Seite 2/24: 25 Zeilen
Seite 3/24: 25 Zeilen
Seite 4/24: 25 Zeilen
Seite 5/24: 25 Zeilen
Seite 6/24: 25 Zeilen
Seite 7/24: 25 Zeilen
Seite 8/24: 25 Zeilen
Seite 9/24: 25 Zeilen
Seite 10/24: 25 Zeilen
Seite 11/24: 25 Zeilen
Seite 12/24: 25 Zeilen
Seite 13/24: 25 Zeilen
Seite 14/24: 25 Zeilen
Seite 15/24: 25 Zeilen
Seite 16/24: 25 Zeilen
Seite 17/24: 25 Zeilen
Seite 18/24: 25 Zeilen
Seite 19/24: 25 Zeilen
Seite 20/24: 25 Zeilen
Seite 21/24: 25 Zeilen
Seite 22/24: 25 Zeilen
Seite 23/24: 25 Zeilen
Seite 24/24: 25 Zeilen
Gesamt gesammelte Zeilen: 600


In [6]:
# DataFrame, Typen bereinigen, CSV speichern

df = pd.DataFrame(all_rows)

int_cols = ["Year", "Wins", "Losses", "OT Losses", "Goals For (GF)", "Goals Against (GA)", "+ / -"]
for col in int_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")

df["Win %"] = pd.to_numeric(df["Win %"], errors="coerce")

display(df.head())
print("Zeilen x Spalten:", df.shape)

df.to_csv("data.csv", index=False)
print("CSV gespeichert als data.csv")


,Team Name,Year,Wins,Losses,OT Losses,Win %,Goals For (GF),Goals Against (GA),+ / -
0,Boston Bruins,1990,44,24,NaN,0.550,299,264,35
1,Buffalo Sabres,1990,31,30,NaN,0.388,292,278,14
2,Calgary Flames,1990,46,26,NaN,0.575,344,263,81
3,Chicago Blackhawks,1990,49,23,NaN,0.613,284,211,73
4,Detroit Red Wings,1990,34,38,NaN,0.425,273,298,-25


Zeilen x Spalten: (600, 9)
CSV gespeichert als data.csv


In [7]:
# Aufgabenfragen mit Pandas beantworten

# Frage1: Wer hatte die meisten "Wins" in 1990, 2000, 2010
def top_wins(year):
    subset = df[df["Year"] == year]
    if subset.empty:
        return None
    idx = subset["Wins"].idxmax()
    return subset.loc[idx, ["Team Name", "Wins"]]

print("Meiste Wins:")
for y in [1990, 2000, 2010]:
    result = top_wins(y)
    if result is None:
        print(f"{y}: keine Daten gefunden")
    else:
        print(f"{y}: {result['Team Name']} mit {int(result['Wins'])} Wins")

# Frage2: Wie viele Teams in 1991, 2001, 2011?

def teams_count(year):
    return df[df["Year"] == year]["Team Name"].nunique()

print("\nAnzahl Teams:")
for y in [1991, 2001, 2011]:
    print(f"{y}: {teams_count(y)} Teams")



Meiste Wins:
1990: Chicago Blackhawks mit 49 Wins
2000: keine Daten gefunden
2010: keine Daten gefunden

Anzahl Teams:
1991: 4 Teams
2001: 0 Teams
2011: 0 Teams
